# The Genesis process

Creating a new game requires the creation of a system and planets, factions and people.

This notebook tests that process as well as documents it. This notebook tests several attributes of the generation process to ensure that the code is sound. 

In [1]:
import numpy as np
import pandas as pd

import sys, os, yaml, ssl, asyncio
import altair as alt

# mapping to the modules that make the app
sys.path.insert(0, "../..")
sys.path.insert(0, "../../app")

%load_ext lab_black

In [2]:
from app import creators
from app import objects

In [3]:
from app.creators import homeworld
from app.creators import universe

An example `data` set for testing. This is the user form that the user submits when creating a new game. 

You can get this from the template at `app\templates\app\creation\genesis.js`


In [4]:
data = {
    "label": "form",
    "name": "worldgenform",
    "objid": "0000000000001",
    "owner": "user.username",
    "username": "user.username",
    "accountid": "0000000000001",
    "conformity": 0.5,
    "constitution": 0.5,
    "literacy": 0.5,
    "aggression": 0.5,
    "num_planets": 4,
    "num_moons": 10,
    "starting_pop": 7,
    "organics": 0.5,
    "minerals": 0.5,
}

You'll also need to configuration files.

In [5]:
conf = creators.universe.configurations.get_configurations()

Each object as an `__init__` function that creates it, however not all data is spawed on init. 

## The Solar System and Planets

In [6]:
system = universe.celestials.System(data)
system

<sytem: ordered; 8691092916686; Gueger>

In [7]:
system.get_data()

{'objid': '8691092916686',
 'name': 'Gueger',
 'label': 'sytem',
 'class': 'ordered',
 'isHomeSystem': True,
 'glat': 18.878,
 'glon': 3.62,
 'gelat': -2.105}

Each object as an `__init__` function that creates it, and populates it with data available at that time. Celestial objects have dependancies on other objects. 

In [8]:
star = objects.celestials.Star(conf["star_config"], system)
star.get_data()

{'name': 'Tle',
 'class': 'G',
 'objid': '8924144735502',
 'label': 'star',
 'radius': 106}

Each object also inherits a `get_fundamentals()` method that ensures that needed default values are present. `get_data()` extends that functionality. This ensures that objects can alwasy interact with the graph. 

In [9]:
star.get_fundimentals()

{'name': 'Tle', 'class': 'G', 'objid': '8924144735502', 'label': 'star'}

Procedurally generated planets chose from a list of potential types in the `conf`. To force a particular kind of outcome, reduce the options in the configuration. 

In [10]:
terrestrial_config = {"terrestrial": conf["planet_config"]["terrestrial"]}
home_planet = objects.celestials.Planet(conf=terrestrial_config, orbiting=star)
home_planet.get_data()

{'name': 'Gertre',
 'class': 'terrestrial',
 'objid': '4452486213974',
 'label': 'planet',
 'radius': 0.089,
 'mass': 0.932,
 'orbitsDistance': 1.117,
 'orbitsId': '8924144735502',
 'orbitsName': 'Tle',
 'isSupportsLife': False,
 'isPopulated': False}

Creating a group of planets using list comprehension. Note that celestial objects have a custom `__repr__` function that makes them easy to manage.

In [11]:
planets = [
    objects.celestials.Planet(conf=conf["planet_config"], orbiting=star)
    for p in range(int(data["num_planets"]) - 1)
]
planets

[<planet: dwarf; 9849777958292; Aunan>,
 <planet: dwarf; 8646991370610; Zaby>,
 <planet: dwarf; 1260030588143; Pur>]

In [12]:
moons = [
    objects.celestials.Moon(conf["moon_config"], planets)
    for p in range(int(data["num_moons"]))
]
moons

[<moon: rocky; 1534660812247; Fusen>,
 <moon: terrestrial; 7230042382801; Essunbi>,
 <moon: ice; 8717834587425; Hillbyriaay>,
 <moon: rocky; 9981760511129; Pesthabay>,
 <moon: ice; 4814605558205; Dongsa>,
 <moon: rocky; 8346992745823; Ahganmas>,
 <moon: ice; 8471613052155; Burpahui>,
 <moon: rocky; 4681485558055; Fieldlarkiykam>,
 <moon: rocky; 4730879842865; Nuemay>,
 <moon: rocky; 2609976509617; Wan>]

Getting the nodes and edges of each by calling the `self.orbiting` propperty.

In [13]:
moons[0].orbiting

<planet: dwarf; 1260030588143; Pur>

In [14]:
home_planet.orbiting

<star: G; 8924144735502; Tle>

Additionally, you can quickly navigate the system by referencing other objects. 

In [15]:
home_planet.orbiting.system

<sytem: ordered; 8691092916686; Gueger>

Getting the nodes for the graph. Sandwich all of the items together and get the data using the same generic function.

In [16]:
all_entities = [system] + [star] + moons + planets + [home_planet]
all_nodes = [b.get_data() for b in all_entities]
pd.DataFrame(all_nodes)

,objid,name,label,class,isHomeSystem,glat,glon,gelat,radius,orbitsId,orbitsName,orbitsDistance,mass,isSupportsLife,isPopulated
0,8691092916686,Gueger,sytem,ordered,True,18.878,3.62,-2.105,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8924144735502,Tle,star,G,NaN,NaN,NaN,NaN,106.000000,NaN,NaN,NaN,NaN,NaN,NaN
2,1534660812247,Fusen,moon,rocky,NaN,NaN,NaN,NaN,0.018597,1260030588143,Pur,0.2280,0.000164,False,False
3,7230042382801,Essunbi,moon,terrestrial,NaN,NaN,NaN,NaN,0.003637,9849777958292,Aunan,0.2890,0.005067,False,False
4,8717834587425,Hillbyriaay,moon,ice,NaN,NaN,NaN,NaN,0.009459,1260030588143,Pur,0.1851,0.024756,False,False
5,9981760511129,Pesthabay,moon,rocky,NaN,NaN,NaN,NaN,0.041583,1260030588143,Pur,0.2600,0.000124,False,False
6,4814605558205,Dongsa,moon,ice,NaN,NaN,NaN,NaN,0.003396,8646991370610,Zaby,0.1121,0.002610,False,False
7,8346992745823,Ahganmas,moon,rocky,NaN,NaN,NaN,NaN,0.001499,8646991370610,Zaby,0.1121,0.000053,False,False
8,8471613052155,Burpahui,moon,ice,NaN,NaN,NaN,NaN,0.008289,1260030588143,Pur,0.1851,0.022834,False,False
9,4681485558055,Fieldlarkiykam,moon,rocky,NaN,NaN,NaN,NaN,0.038036,1260030588143,Pur,0.1851,0.000334,False,False


Getting the edge values to update the graph: 

In [17]:
orbiting_bodies = [home_planet] + planets + moons

orbiting_edges = [i.get_orbits_edge() for i in orbiting_bodies]

pd.DataFrame(orbiting_edges)

,node1,node2,label,orbit_distance
0,4452486213974,8924144735502,orbits,1.1170
1,9849777958292,8924144735502,orbits,45.1610
2,8646991370610,8924144735502,orbits,39.8650
3,1260030588143,8924144735502,orbits,34.2290
4,1534660812247,1260030588143,orbits,0.0430
5,7230042382801,9849777958292,orbits,0.1550
6,8717834587425,1260030588143,orbits,0.0001
7,9981760511129,1260030588143,orbits,0.0750
8,4814605558205,8646991370610,orbits,0.0001
9,8346992745823,8646991370610,orbits,0.0001


Check the orbiting logic

In [18]:
[
    f"{i.type} {i.label} orbits a {i.orbiting.type} {i.orbiting.label}"
    for i in orbiting_bodies
]

['terrestrial planet orbits a G star',
 'dwarf planet orbits a G star',
 'dwarf planet orbits a G star',
 'dwarf planet orbits a G star',
 'rocky moon orbits a dwarf planet',
 'terrestrial moon orbits a dwarf planet',
 'ice moon orbits a dwarf planet',
 'rocky moon orbits a dwarf planet',
 'ice moon orbits a dwarf planet',
 'rocky moon orbits a dwarf planet',
 'ice moon orbits a dwarf planet',
 'rocky moon orbits a dwarf planet',
 'rocky moon orbits a dwarf planet',
 'rocky moon orbits a dwarf planet']

In [19]:
system_bodies = orbiting_bodies + [star]

system_edges = [i.get_in_system_edge() for i in system_bodies]

pd.DataFrame(system_edges)

,node1,node2,label
0,4452486213974,8691092916686,isIn
1,9849777958292,8691092916686,isIn
2,8646991370610,8691092916686,isIn
3,1260030588143,8691092916686,isIn
4,1534660812247,8691092916686,isIn
5,7230042382801,8691092916686,isIn
6,8717834587425,8691092916686,isIn
7,9981760511129,8691092916686,isIn
8,4814605558205,8691092916686,isIn
9,8346992745823,8691092916686,isIn


### Scanning the homeworld
the homeworld already has some resources known. `scan_body()` is inherited by the base object. So everyone should be able to do it. 

In [20]:
home_planet.scan_body()
home_planet.resources

[<resource: 0353831835892; Organic>,
 <resource: 5006127418288; Common Minerals>,
 <resource: 7395522610414; Rare Minerals>,
 <resource: 7160228082655; Water>]

In [21]:
pd.DataFrame([i.get_data() for i in home_planet.resources])

,name,objid,label,volume,max_volume,description,replenish_rate
0,Organic,0353831835892,resource,996,996,bilogical material that can be consumed by pops,10.0
1,Common Minerals,5006127418288,resource,104,104,Iron and other common material used in constru...,NaN
2,Rare Minerals,7395522610414,resource,45,45,"lithium, silver and other rare minerals used i...",NaN
3,Water,7160228082655,resource,11027,11027,"H2O ready to be consumed, either frozen or in ...",NaN


In [22]:
pd.DataFrame([i.get_location_edge() for i in home_planet.resources])

,node1,node2,label
0,4452486213974,0353831835892,has
1,4452486213974,5006127418288,has
2,4452486213974,7395522610414,has
3,4452486213974,7160228082655,has


In [23]:
[
    f"{i.location.type} {i.location.label} has {i.volume} {i.name}"
    for i in home_planet.resources
]

['terrestrial planet has 996 Organic',
 'terrestrial planet has 104 Common Minerals',
 'terrestrial planet has 45 Rare Minerals',
 'terrestrial planet has 11027 Water']

# The Full Automated Process

Usefull as QA, to ensure that the process will run end-to-end

In [24]:
homesystem_data = universe.build_homeSystem(data, username="notebook")
pd.DataFrame(homesystem_data["nodes"])

,objid,name,label,class,isHomeSystem,glat,glon,gelat,radius,orbitsId,...,accountid,conformity,constitution,literacy,aggression,num_planets,num_moons,starting_pop,organics,minerals
0,6841850514561,Bhatinlegun,sytem,ordered,True,2.153,-10.164,-0.194,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0942355528963,Gun,star,G,NaN,NaN,NaN,NaN,106.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5319107306011,Quagunis,moon,ice,NaN,NaN,NaN,NaN,0.178419,5205068490871,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8384226142042,Karmerter,moon,ice,NaN,NaN,NaN,NaN,0.023244,3431176345279,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9149774422394,Din,moon,terrestrial,NaN,NaN,NaN,NaN,0.013013,4914251675710,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,8224251888768,Franil,moon,rocky,NaN,NaN,NaN,NaN,0.025871,4403122776966,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,9949942137176,Ervilcen,moon,rocky,NaN,NaN,NaN,NaN,0.004555,4403122776966,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,5321943267619,Teinheightsniashi,moon,ice,NaN,NaN,NaN,NaN,0.027571,3431176345279,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,5538658987159,Pia,moon,rocky,NaN,NaN,NaN,NaN,0.173536,3431176345279,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2905261083225,Ildorfgrebon,moon,terrestrial,NaN,NaN,NaN,NaN,0.005001,4914251675710,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
print(
    f"{len(homesystem_data['edges'])}  total edges for {len(homesystem_data['nodes'])} objects"
)
pd.DataFrame(homesystem_data["edges"]).groupby("label").count()

37  total edges for 22 objects


,node1,node2,orbit_distance
label,,,
createdFrom,1,1,0
has,4,4,0
isIn,16,16,0
orbits,15,15,15
submitted,1,1,0


### Rulse For Ontology:
here is a point to check that the ontology matches this format:
* everything camelCase (starting on lower)
* Don't reference the subject in the edge label (has not hasResource)
* Every node is a Class that can be found in `objects`
* Form and Account are the exception

# The planet Surface

Entirely separate processes, the homeworld makes the population on a planet that already exists. 

In [26]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [27]:
spec = objects.species.Species(data)
spec

<species: None; 5017782364858; Lerguelong>

In [28]:
pops = [objects.population.Pop(spec) for i in range(int(data["starting_pop"]))]
pops_df = pd.DataFrame([p.get_data() for p in pops])
pops_df

,name,objid,label,conformity,literacy,aggression,constitution,health,isInFaction,industry,wealth,factionLoyalty,isIdle
0,unnamed,1725094570539,pop,0.391,0.593,0.568,0.493,0.5,None,0.5305,0.56175,0.513,True
1,unnamed,0661587749240,pop,0.509,0.680,0.454,0.569,0.5,None,0.5115,0.59575,0.484,True
2,unnamed,9765425958616,pop,0.512,0.544,0.474,0.439,0.5,None,0.4565,0.50025,0.612,True
3,unnamed,7360924572408,pop,0.564,0.487,0.616,0.428,0.5,None,0.5220,0.50450,0.633,True
4,unnamed,1768984315923,pop,0.633,0.532,0.488,0.593,0.5,None,0.5405,0.53625,0.608,True
5,unnamed,6072035642591,pop,0.569,0.422,0.383,0.480,0.5,None,0.4315,0.42675,0.660,True
6,unnamed,1980869252349,pop,0.441,0.527,0.299,0.397,0.5,None,0.3480,0.43750,0.529,True


Populations (_pops_) are born with stats that are within normal distribution for it's species. They are not in a faction and don't have names. The faction will help to determine their names. 

In [29]:
n_steps = 6
n_factions = homeworld.get_n_factions(n_steps, float(data["conformity"]))
n_factions

2

Using conformity to get the number of factions. Higher conformity, fewer factions. 

In [30]:
starting_attributes = ["conformity", "literacy", "aggression", "constitution"]

kmeans = KMeans(n_clusters=n_factions).fit(
    pops_df[[c for c in pops_df.columns if c in starting_attributes]]
)
kmeans

KMeans(n_clusters=2)

In [31]:
factions = [objects.population.Faction(i) for i in range(kmeans.n_clusters)]
factions

[<faction: None; 7043075192983; Hedo>,
 <faction: None; 3542569953657; Pingtesnorth>]

In [32]:
kmeans.labels_

array([1, 1, 1, 1, 1, 0, 0])

In [33]:
for i, n in enumerate(kmeans.labels_):
    pops[i].set_faction(factions[n])
    print(f"pop: {pops[i]} belongs to faction: {factions[n]}")

pop: <pop: pop; 1725094570539; Pingtesnorth Reigorskfran> belongs to faction: <faction: None; 3542569953657; Pingtesnorth>
pop: <pop: pop; 0661587749240; Pingtesnorth Geenharhui> belongs to faction: <faction: None; 3542569953657; Pingtesnorth>
pop: <pop: pop; 9765425958616; Pingtesnorth La> belongs to faction: <faction: None; 3542569953657; Pingtesnorth>
pop: <pop: pop; 7360924572408; Pingtesnorth Can> belongs to faction: <faction: None; 3542569953657; Pingtesnorth>
pop: <pop: pop; 1768984315923; Pingtesnorth Sel> belongs to faction: <faction: None; 3542569953657; Pingtesnorth>
pop: <pop: pop; 6072035642591; Hedo Hamneudi> belongs to faction: <faction: None; 7043075192983; Hedo>
pop: <pop: pop; 1980869252349; Hedo Atmekha> belongs to faction: <faction: None; 7043075192983; Hedo>


The name of the population should match the name of the faction. 

In [34]:
pops_df = pd.DataFrame([p.get_data() for p in pops])
pops_df

,name,objid,label,conformity,literacy,aggression,constitution,health,isInFaction,industry,wealth,factionLoyalty,isIdle
0,Pingtesnorth Reigorskfran,1725094570539,pop,0.391,0.593,0.568,0.493,0.5,3542569953657,0.5305,0.56175,0.513,True
1,Pingtesnorth Geenharhui,0661587749240,pop,0.509,0.680,0.454,0.569,0.5,3542569953657,0.5115,0.59575,0.484,True
2,Pingtesnorth La,9765425958616,pop,0.512,0.544,0.474,0.439,0.5,3542569953657,0.4565,0.50025,0.612,True
3,Pingtesnorth Can,7360924572408,pop,0.564,0.487,0.616,0.428,0.5,3542569953657,0.5220,0.50450,0.633,True
4,Pingtesnorth Sel,1768984315923,pop,0.633,0.532,0.488,0.593,0.5,3542569953657,0.5405,0.53625,0.608,True
5,Hedo Hamneudi,6072035642591,pop,0.569,0.422,0.383,0.480,0.5,7043075192983,0.4315,0.42675,0.660,True
6,Hedo Atmekha,1980869252349,pop,0.441,0.527,0.299,0.397,0.5,7043075192983,0.3480,0.43750,0.529,True


Just as the pops reference the faction, the faction has a list of it's populations. 

In [35]:
factions[0].pops

[<pop: pop; 6072035642591; Hedo Hamneudi>,
 <pop: pop; 1980869252349; Hedo Atmekha>]

Creating a PCA to separate the factions by distance. 

In [39]:
if n_factions >= 2:
    # using PCA to set populations on map:

    # PCA Part
    pca = PCA(n_components=2)
    X_r = pca.fit(kmeans.cluster_centers_).transform(kmeans.cluster_centers_)
    for i, f in enumerate(factions):
        f.pca_explained_variance_ratio = pca.explained_variance_ratio_
        f.lat = np.round(X_r[i][0], 3)
        f.long = np.round(X_r[i][1], 3)
else:
    # Only one faction, the lat and long is 0,0
    for i, f in enumerate(factions):
        f.lat = 0
        f.long = 0

In [41]:
pca.explained_variance_ratio_

array([1.00000000e+00, 3.14478388e-31])